<a href="https://colab.research.google.com/github/NinadShegokar/Fine-Tune/blob/main/bnb_4bit_training_with_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install pandas numpy evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/38

In [2]:
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "l3cube-pune/marathi-bert-v2"



# Configuration for BitsAndBytes quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-bert-v2")
model = AutoModelForMaskedLM.from_pretrained("l3cube-pune/marathi-bert-v2")

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

In [3]:
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "l3cube-pune/marathi-bert-v2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at l3cube-pune/marathi-bert-v2 and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    # target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 294912 || all params: 195485002 || trainable%: 0.15086170140049926


In [7]:
import pandas as pd

url = "/content/final_marathi_data.csv"

df = pd.read_csv(url)

print(df.head())

   Unnamed: 0                                               text  label
0           0          होता होता राहीलेला  निवडणूक मारो मर्ज़ीभई     -1
1           1                     खरा लखोबा तर हा बोबडाच आहे         -1
2           2  मुंबईतील घाटकोपरमध्ये धुळवड खेळून घरी परतलेलं ...     -1
3           3      वेडाबाई भूतकाळ बघ लोक शेन घालतात आणी दांडा ही     -1
4           4   कुर्ला वाहतुक विभागाला फक्त हे पाठवले जाते पण...     -1


In [8]:
from datasets import Dataset

data = Dataset.from_pandas(df)
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/48114 [00:00<?, ? examples/s]

In [10]:
from datasets import Dataset
import transformers
from sklearn.model_selection import train_test_split

# Assuming data["text"] is a list of strings
# Assuming data["label"] is a list of labels corresponding to the text
# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data["text"], data["label"], test_size=0.2, random_state=42)

# Create training and testing datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Tokenize the text data using the tokenizer for both training and testing datasets
train_tokenized_dataset = train_dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True)
test_tokenized_dataset = test_dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True)

# Pass the tokenized datasets to the Trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,  # Use test dataset for evaluation
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Map:   0%|          | 0/38491 [00:00<?, ? examples/s]

Map:   0%|          | 0/9623 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from datasets import Dataset
import transformers
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Assuming data["text"] is a list of strings
dataset = Dataset.from_dict({"text": data["text"]})

# Tokenize the text data using the tokenizer
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length"), batched=True)

# Pass the tokenized dataset to the Trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [11]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [12]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch

# Define the device for computation (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming you have already split your dataset into testing set
# test_texts and true_labels represent the testing data

# Tokenize the test data using the same tokenizer used during fine-tuning
tokenized_test_texts = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")

# Move the tokenized data to the appropriate device (CPU/GPU) for inference
tokenized_test_texts = {key: value.to(device) for key, value in tokenized_test_texts.items()}

# Generate predictions using your fine-tuned model
with torch.no_grad():
    outputs = model(**tokenized_test_texts)
    predicted_labels = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Compute accuracy using sklearn's accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)

print("Accuracy:", accuracy)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import requests
from io import StringIO

# URL of the test dataset
url = "https://raw.githubusercontent.com/l3cube-pune/MarathiNLP/main/L3Cube-MahaSent-MD/MahaSent_All/MahaSent_All_Test.csv"

# Read the test dataset
response = requests.get(url)
test_df = pd.read_csv(StringIO(response.text))

# Assuming your test dataset has 'text' and 'label' columns
test_texts = test_df['text'].tolist()
true_labels = test_df['label'].tolist()

# Tokenize the test data (replace this with the appropriate tokenizer and tokenization method)
tokenized_test_texts = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")

# Generate predictions using your fine-tuned model
with torch.no_grad():
    outputs = model(**tokenized_test_texts)
    predicted_labels = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Compute accuracy using sklearn's accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)

print("Accuracy:", accuracy)

In [ ]:
text = " "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))